In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Entertainment_Music/model_outputs/informal.nmt_copy
/kaggle/input/Entertainment_Music/model_outputs/formal.nmt_baseline
/kaggle/input/Entertainment_Music/model_outputs/formal.nmt_combined
/kaggle/input/Entertainment_Music/model_outputs/informal.nmt_combined
/kaggle/input/Entertainment_Music/model_outputs/informal.rule_based
/kaggle/input/Entertainment_Music/model_outputs/informal.pbmt
/kaggle/input/Entertainment_Music/model_outputs/formal.nmt_copy
/kaggle/input/Entertainment_Music/model_outputs/formal.pbmt
/kaggle/input/Entertainment_Music/model_outputs/informal.nmt_baseline
/kaggle/input/Entertainment_Music/model_outputs/formal.rule_based
/kaggle/input/Entertainment_Music/tune/formal
/kaggle/input/Entertainment_Music/tune/informal.ref2
/kaggle/input/Entertainment_Music/tune/formal.ref2
/kaggle/input/Entertainment_Music/tune/formal.ref3
/kaggle/input/Entertainment_Music/tune/formal.ref1
/kaggle/input/Entertainment_Music/tune/informal
/kaggle/input/Entertainment_Music/tune

In [2]:
!pip uninstall -y transformers
!pip install transformers
!pip install accelerate -U

Found existing installation: transformers 4.39.3
Uninstalling transformers-4.39.3:
  Successfully uninstalled transformers-4.39.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 28.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 83.5 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.6 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.28.0
    Uninstalling accelerate-0.28.0:
      Successfully uninstalled accelerate-0.28.0


In [3]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model_checkpoint = "t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

2024-04-22 03:36:06.791545: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 03:36:06.791644: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 03:36:06.891027: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
     fp16=True,
    push_to_hub=True,
)

In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords

# Initialize Snowball stemmer
stemmer = SnowballStemmer("english")

# Initialize stopwords
stop_words = set(stopwords.words('english'))

# Function to apply stemming to a sentence using Snowball stemmer
def apply_stemming(sentence):
    tokens = word_tokenize(sentence)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

# Function to replace terms with POS tags based on weights from the model
def replace_with_pos_tags(sentence, weights, is_formal):
    tokens = word_tokenize(sentence)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]  # Stem tokens
    pos_tags = pos_tag(tokens)  # Use original tokens for POS tagging
    replaced_sentence = []

    # Calculate N as described
    N = len(stemmed_tokens) // 4

    # Filter out stop words and stem them
    tokens_without_stopwords = [stemmer.stem(token) for token in tokens if token.lower() not in stop_words]

    # Sort stemmed tokens by weight
    sorted_tokens = sorted(tokens_without_stopwords, key=lambda x: abs(weights.get(x, 0)), reverse=True)

    # Extract N tokens with highest absolute weights
    top_N_tokens = sorted_tokens[:N]

    for token, tag in pos_tags:
        stemmed_token = stemmer.stem(token)
        if stemmed_token in top_N_tokens:
            weight = abs(weights.get(stemmed_token, 0))
            if (is_formal and weight >= 0.01) or (not is_formal and weight >= 0.02):
                replaced_sentence.append(tag)
            else:
                replaced_sentence.append(token)
        else:
            replaced_sentence.append(token)

    return ' '.join(replaced_sentence)

def read_data(file_path):
    with open(file_path, "r", encoding="latin-1") as file:
        sentences = file.readlines()
    return [apply_stemming(sentence.strip()) for sentence in sentences]

# Paths to data files
train_formal_path_1 = "/kaggle/input/Entertainment_Music/train/formal"
train_informal_path_1 = "/kaggle/input/Entertainment_Music/train/informal"
test_formal_path_1 = "/kaggle/input/Entertainment_Music/test/formal"
test_informal_path_1 = "/kaggle/input/Entertainment_Music/test/informal"

train_formal_path_2 = "/kaggle/input/Family_Relationships/train/formal"
train_informal_path_2 = "/kaggle/input/Family_Relationships/train/informal"
test_formal_path_2 = "/kaggle/input/Family_Relationships/test/formal"
test_informal_path_2 = "/kaggle/input/Family_Relationships/test/informal"

X_train = (read_data(train_formal_path_1) + read_data(train_formal_path_2) +
           read_data(train_informal_path_1) + read_data(train_informal_path_2))
Y_train = [0] * (len(read_data(train_formal_path_1)) + len(read_data(train_formal_path_2))) + \
          [1] * (len(read_data(train_informal_path_1)) + len(read_data(train_informal_path_2)))  # Labels for training data

X_test = read_data(test_formal_path_1) + read_data(test_formal_path_2) + \
         read_data(test_informal_path_1) + read_data(test_informal_path_2)
Y_test = [0] * (len(read_data(test_formal_path_1)) + len(read_data(test_formal_path_2))) + \
         [1] * (len(read_data(test_informal_path_1)) + len(read_data(test_informal_path_2)))  # Labels for test data


# Initialize TfidfVectorizer with n-grams
vectorizer = TfidfVectorizer(max_features=200000, ngram_range=(1, 3), min_df=1, max_df=0.85)


# Initialize StandardScaler for scaling the data
scaler = StandardScaler(with_mean=False)

# Create a Pipeline for preprocessing and model training
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', LogisticRegression( max_iter=5000))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, Y_train)

# Extract feature names from the vectorizer
# feature_names = vectorizer.get_feature_names()
feature_names = vectorizer.get_feature_names_out()


# Extract coefficients (weights) from the logistic regression model
weights = dict(zip(feature_names, pipeline.named_steps['classifier'].coef_[0]))

# Set threshold for replacing terms with POS tags
# threshold = 0.2

# Replace terms with POS tags in the training data itself
X_train_transformed = [replace_with_pos_tags(X_train[i], weights, not(Y_train[i])) for i  in range(len(X_train))]

# Print out the original and transformed sentences
# for original_sentence, transformed_sentence in zip(X_train, X_train_transformed):
#     print("Original Sentence:", original_sentence)
#     print("Transformed Sentence:", transformed_sentence)
#     print()  # Add a newline for clarity

# Make predictions on the test data
predicted_probabilities = pipeline.predict_proba(X_test)

# Define a new threshold
new_threshold = 0.42  # Example threshold, you can adjust it according to your needs

# Modify predictions based on the new threshold
predicted_labels = (predicted_probabilities[:, 1] >= new_threshold).astype(int)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)
print("Accuracy:", accuracy)

# Compute confusion matrix
cm = confusion_matrix(Y_test, predicted_labels)
print("Confusion Matrix:")
print(cm)


Accuracy: 0.795009280263972
Confusion Matrix:
[[1541  560]
 [ 434 2314]]


In [11]:
def replace_with_pos_tags(sentence, weights, is_formal):
    tokens = word_tokenize(sentence)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]  # Stem tokens
    pos_tags = pos_tag(tokens)  # Use original tokens for POS tagging
    replaced_sentence = []

    # Calculate N as described
    N = len(stemmed_tokens) // 4

    # Filter out stop words and stem them
    tokens_without_stopwords = [stemmer.stem(token) for token in tokens if token.lower() not in stop_words]

    # Sort stemmed tokens by weight
    sorted_tokens = sorted(tokens_without_stopwords, key=lambda x: abs(weights.get(x, 0)), reverse=True)

    # Extract N tokens with highest absolute weights
    top_N_tokens = sorted_tokens[:N]

    for token, tag in pos_tags:
        stemmed_token = stemmer.stem(token)
        if stemmed_token in top_N_tokens:
            weight = abs(weights.get(stemmed_token, 0))
            if (is_formal and weight >= 0.01) or (not is_formal and weight >= 0.02):
                replaced_sentence.append(tag)
            else:
                replaced_sentence.append(token)
        else:
            replaced_sentence.append(token)

    return ' '.join(replaced_sentence)


In [18]:
def read_data(file_path):
    with open(file_path, "r", encoding="latin-1") as file:
        sentences = file.readlines()
    return [sentence.strip() for sentence in sentences]

In [19]:
X_train = (read_data(train_formal_path_1) + read_data(train_formal_path_2) +
           read_data(train_informal_path_1) + read_data(train_informal_path_2))
Y_train = [0] * (len(read_data(train_formal_path_1)) + len(read_data(train_formal_path_2))) + \
          [1] * (len(read_data(train_informal_path_1)) + len(read_data(train_informal_path_2)))  # Labels for training data

X_test = read_data(test_formal_path_1) + read_data(test_formal_path_2) + \
         read_data(test_informal_path_1) + read_data(test_informal_path_2)
Y_test = [0] * (len(read_data(test_formal_path_1)) + len(read_data(test_formal_path_2))) + \
         [1] * (len(read_data(test_informal_path_1)) + len(read_data(test_informal_path_2)))

In [23]:
X_train_transformed = [replace_with_pos_tags(X_train[i], weights, not(Y_train[i])) for i  in range(len(X_train))]


In [12]:
new=replace_with_pos_tags("There's nothing he needs to change.", weights, True)

new

"There 's nothing he VBZ to VB ."

In [61]:
X_test=[apply_stemming(sentence) for sentence in X_test]

In [63]:
# X_test

In [64]:
predicted_probabilities = pipeline.predict_proba(X_test)

# Define a new threshold
new_threshold = 0.45  # Example threshold, you can adjust it according to your needs

# Modify predictions based on the new threshold
predicted_labels = (predicted_probabilities[:, 1] >= new_threshold).astype(int)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)
print("Accuracy:", accuracy)

# Compute confusion matrix
cm = confusion_matrix(Y_test, predicted_labels)
print("Confusion Matrix:")
print(cm)

Accuracy: 0.7925345432047844
Confusion Matrix:
[[1631  470]
 [ 536 2212]]


In [44]:
pipeline


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_df=0.85, max_features=200000,
                                 ngram_range=(1, 3))),
                ('classifier', LogisticRegression(max_iter=5000))])

In [39]:
weights["crap"]

4.204547427074

In [20]:
X_test_transformed = [replace_with_pos_tags(X_test[i], weights, not(Y_test[i])) for i  in range(len(X_test))]

In [37]:
# X_test_transformed

In [25]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00a 0:00:01


In [26]:
from evaluate import load
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a078449309afb09dadb855a745f6a9a4604219d9e1ac479840a87cce9d1a69d3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [27]:
metric1 = load("rouge")

In [28]:
metric2 = load("bleu")

In [29]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [30]:
def read_data(file_path):
    with open(file_path, "r", encoding="latin-1") as file:
        sentences = file.readlines()
    return [sentence.strip() for sentence in sentences]


# Paths to data files
train_formal_path_1 = "/kaggle/input/Entertainment_Music/train/formal"
train_informal_path_1 = "/kaggle/input/Entertainment_Music/train/informal"
test_formal_path_1 = "/kaggle/input/Entertainment_Music/test/formal"
test_informal_path_1 = "/kaggle/input/Entertainment_Music/test/informal"

train_formal_path_2 = "/kaggle/input/Family_Relationships/train/formal"
train_informal_path_2 = "/kaggle/input/Family_Relationships/train/informal"
test_formal_path_2 = "/kaggle/input/Family_Relationships/test/formal"
test_informal_path_2 = "/kaggle/input/Family_Relationships/test/informal"

X_train_formal = (read_data(train_formal_path_1) + read_data(train_formal_path_2) )
X_train_informal = read_data(train_informal_path_1) + read_data(train_informal_path_2)


X_test_formal = read_data(test_formal_path_1) + read_data(test_formal_path_2)
X_test_informal = read_data(test_informal_path_1) + read_data(test_informal_path_2)


In [31]:
X_train_formal_transformed=X_train_transformed[:len(X_train)//2]

In [32]:
X_test_formal_transformed=X_test_transformed[0:len(X_test_formal)]

In [33]:
train_formal_df = pd.DataFrame({ 'POS': X_train_formal_transformed, 'formal': X_train_formal,})

In [34]:
train_formal_df["formal"][2:3]

2    I watch it everyday, my favorite charachter is...
Name: formal, dtype: object

In [35]:
test_formal_df = pd.DataFrame({ 'POS': X_test_formal_transformed, 'formal': X_test_formal,})

In [69]:
test_formal_df

,POS,formal
0,I like NNP and Blue music .,I like Rhythm and Blue music.
1,There 's nothing he VBZ to VB .,There's nothing he needs to change.
2,It does not VB .,It does not exist.
3,Mine is book by Steve Martin called POS NN of ...,Mine is book by Steve Martin called 'The Pleas...
4,What VBZ a mosquitoo from a NN ?,What differentiates a mosquitoo from a blonde?
...,...,...
2096,The world MD be happier if NNS knew what NNS w...,The world would be happier if men knew what wo...
2097,You want to have oral NN with another NN ?,You want to have oral sex with another man?
2098,I VBP you not marry .,I prefer you not marry.
2099,"Yes I am a NN , RB I do not RB have to pay .","Yes I am a male, therefore I do not really hav..."


In [67]:
train_formal_df

,POS,formal
0,"The In-Laws NN is n't a JJ NN , but it 's NN .","The In-Laws movie isn't a holiday movie, but i..."
1,I do n't VB that page gave me NNS .,I don't think that page gave me viruses.
2,"I watch it JJ , my JJ charachter is Inuasha .","I watch it everyday, my favorite charachter is..."
3,Funbrain.com and runescape.com are JJ for fami...,Funbrain.com and runescape.com are great for f...
4,He was on the Late Night NN with Conan O'Brien...,He was on the Late Night show with Conan O'Bri...
...,...,...
104557,"Of NN , it depends on what type of NN you are ...","Of course, it depends on what type of relation..."
104558,IN a sign that say `` NNP ! '',"Wear a sign that say ""Hi!"""
104559,I do not IN when NNS play games with me .,I do not like when guys play games with me.
104560,How JJ are you ?,How old are you?


In [ ]:
test_formal_df

In [ ]:
print("hello")

In [45]:
import pandas as pd

# Assuming you already have X_train_formal_transformed and X_train_formal defined
train_formal_df = pd.DataFrame({'POS': X_train_formal_transformed, 'formal': X_train_formal})

def preprocess_function(row):
    inputs = row["POS"] + "</s>"
    model_inputs = tokenizer(inputs, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=row["formal"], truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset_formal_train = train_formal_df.apply(preprocess_function, axis=1)
tokenized_dataset_formal_test = test_formal_df.apply(preprocess_function, axis=1)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [46]:
from transformers import AutoModelForSeq2SeqLM


In [47]:
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(tokenized_dataset_formal_train, test_size=0.2, random_state=42)


In [52]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [73]:
import nltk
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    preds=[apply_stemming(sentence) for sentence in  decoded_preds]

    # Use logistic regression model to predict formality of decoded sentences
    predicted_formality = pipeline.predict(preds)

    # Calculate accuracy based on logistic regression model's predictions
    total_samples = len(decoded_preds)
    correct_predictions = sum(predicted_formality == 0)  # Count how many were predicted as formal (0)
    accuracy_log_reg = correct_predictions / total_samples

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric1.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    # Add accuracy based on logistic regression model's predictions
    result["accuracy_log_reg"] = accuracy_log_reg
    
    print(result)

    return {k: round(v, 4) for k, v in result.items()}


In [54]:
train_dataset = train_data.to_frame().to_dict(orient='records')
val_dataset = val_data.to_frame().to_dict(orient='records')

In [55]:
train_dataset1 = [v for d in train_dataset for k, v in d.items()]
val_dataset1=[v for d in val_dataset for k, v in d.items()]

In [27]:
print(train_dataset1[:10])

[{'input_ids': [230, 3, 88, 3, 22086, 956, 833, 46, 32, 189, 388, 3, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [852, 3, 88, 19, 4134, 430, 388, 5, 1]}, {'input_ids': [27, 333, 3001, 5333, 1605, 14228, 77, 29, 23, 3, 12111, 3, 18450, 107, 28, 82, 4996, 17, 40, 108, 115, 40, 3, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [27, 333, 12, 10065, 1605, 14883, 11113, 37, 1908, 32, 107, 28, 82, 385, 19665, 5, 1]}, {'input_ids': [24, 3, 22086, 956, 59, 1282, 3, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [466, 19, 59, 1282, 5, 1]}, {'input_ids': [27, 584, 11165, 4974, 63, 36, 75, 2064, 3, 88, 19, 548, 23, 5295, 3, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [27, 2994, 11790, 15, 63, 250, 3, 88, 19, 182, 5295, 5, 1]}, {'input_ids': [1687, 3, 6, 1028, 15, 9, 7, 3, 6, 20595, 3, 6, 9867, 3, 6, 11, 554, 11260, 

In [56]:
from huggingface_hub import notebook_login

notebook_login()

In [74]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset1 ,
    eval_dataset=val_dataset1,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [55]:
test_dataset1 = tokenized_dataset_formal_test.to_frame().to_dict(orient='records')

In [56]:
test2=[v for d in test_dataset1 for k, v in d.items()]

In [76]:
test2[:2]
# train_dataset1[:2]

[{'input_ids': [27, 584, 11165, 10638, 11, 1692, 723, 3, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [27, 114, 3, 27224, 11, 2419, 723, 5, 1]},
 {'input_ids': [262, 4, 3, 31, 7, 59, 107, 3, 88, 174, 12, 3, 75, 9270, 3, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [290, 31, 7, 1327, 3, 88, 523, 12, 483, 5, 1]}]

In [61]:
import torch

In [77]:
test_predictions = trainer.predict(test2)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars

In [139]:
b=output.numpy()
b,a

(array([[   0,  696, 3785, 1499, 1550,  270,    5,    1]]),
 array([    0,    27,   777, 10638,    11,  1692,   723,     5,     1,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0]))

In [141]:
a=test_predictions.predictions[0]
tokenizer.decode(b[0], skip_special_tokens=True)

'Your input text goes here.'

In [123]:
from transformers import TFAutoModelForSeq2SeqLM

# Define the directory where the model is saved
saved_model_dir = "/kaggle/working/saved_model"

# Load the saved model
loaded_model = TFAutoModelForSeq2SeqLM.from_pretrained(saved_model_dir)


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [124]:
from transformers import AutoModelForSeq2SeqLM

# Load the saved model
loaded_model = AutoModelForSeq2SeqLM.from_pretrained(output_dir)

In [131]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(output_dir)

# Tokenize input text
input_text = "Your input text goes here."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Generate predictions
output = loaded_model.generate(input_ids)
output

tensor([[   0,  696, 3785, 1499, 1550,  270,    5,    1]])

In [111]:
trainer

In [ ]:
!pip install

In [75]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Accuracy Log Reg
1,0.568300,0.528100,84.057900,71.563600,83.979800,83.990400,14.266400,0.747400
2,0.544900,0.519130,84.207800,71.795600,84.120700,84.131300,14.271000,0.749600
3,0.534300,0.514250,84.308300,72.002000,84.228000,84.237600,14.279400,0.753000
4,0.521900,0.511729,84.350200,72.089400,84.269200,84.277900,14.284500,0.752600
5,0.517900,0.510852,84.371500,72.107800,84.288400,84.297500,14.280100,0.754400


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

{'rouge1': 84.05785061785366, 'rouge2': 71.56357594133377, 'rougeL': 83.97981985363651, 'rougeLsum': 83.99037070366552, 'gen_len': 14.266389327212739, 'accuracy_log_reg': 0.7474298283364414}


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

{'rouge1': 84.20782128327714, 'rouge2': 71.7956439443813, 'rougeL': 84.1206635824782, 'rougeLsum': 84.13133374942984, 'gen_len': 14.270979773346722, 'accuracy_log_reg': 0.7496294171089752}


...

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

{'rouge1': 84.35022059345305, 'rouge2': 72.08942292275557, 'rougeL': 84.26918500427345, 'rougeLsum': 84.27788607004096, 'gen_len': 14.284464208865298, 'accuracy_log_reg': 0.752641897384402}


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

{'rouge1': 84.37152575886486, 'rouge2': 72.10782428836092, 'rougeL': 84.28838568624523, 'rougeLsum': 84.29751984774867, 'gen_len': 14.28011284846746, 'accuracy_log_reg': 0.7543633146846459}


TrainOutput(global_step=13075, training_loss=0.5423579893969213, metrics={'train_runtime': 11215.9199, 'train_samples_per_second': 37.29, 'train_steps_per_second': 1.166, 'total_flos': 2.04372631206144e+16, 'train_loss': 0.5423579893969213, 'epoch': 5.0})

In [81]:
print("training complete")

training complete


In [82]:
print("testing")

testing


In [83]:
print("testing")

testing


In [90]:
trainer.push_to_hub("Pushparaj20/formal_transfer")

CommitInfo(commit_url='https://huggingface.co/Pushparaj20/t5-base-finetuned/commit/44b89c715b9a8a5b1e70ed3ad12450c3b31eb150', commit_message='Pushparaj20/formal_transfer', commit_description='', oid='44b89c715b9a8a5b1e70ed3ad12450c3b31eb150', pr_url=None, pr_revision=None, pr_num=None)

In [91]:
from transformers import AutoModelForSeq2SeqLM

# Load the model from the model hu
model = AutoModelForSeq2SeqLM.from_pretrained("Pushparaj20/t5-base-finetuned")

In [86]:
test_dataset1 = tokenized_dataset_formal_test.to_frame().to_dict(orient='records')

In [87]:
test2=[v for d in test_dataset1 for k, v in d.items()]

In [45]:
t=test2[0]["input_ids"]

In [46]:
t

[27, 584, 11165, 10638, 11, 1692, 723, 3, 5, 1, 1]

In [88]:
input_text = "Your input text goes here."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
print(input_ids)

# Generate predictions
output = model.generate(input_ids)
b=output.numpy()
tokenizer.decode(b[0], skip_special_tokens=True)

tensor([[ 696, 3785, 1499, 1550,  270,    5,    1]])


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Your input text goes here.'

'Your input text goes here.'

In [83]:
new=[]

In [97]:
for i in range(0,10):
    o=model.generate(input_ids=torch.tensor([test2[i]["input_ids"]]))
    new.append(o)
    
    


In [100]:
o

tensor([[    0,    27,   133,   167,   952,    59,  2902,    21,   376,     6,
          2199,    27,   317, 31297,   133,    36,     8,   167,  5250,   166]])

In [103]:
# decoded_outputs = tokenizer.batch_decode(new, skip_special_tokens=True)

# Print the decoded outputs
for output in new:
    print(tokenizer.batch_decode(output, skip_special_tokens=True))

['I enjoy rhythm and blues music.']
["There's nothing he needs to change."]
['It does not exist.']
["Mine is book by Steve Martin called 'The Pleasure of My Company'."]
['What differentiates a mosquitoo from a blonde?']
["They're very attractive. Also, that's a good song."]
['I do not think Beyonce can sing, dance, or perform. You mentioned Rih']
['I was unaware that you were in law enforcement, as well.']
["I call to say 'I love you"]
['I would most likely not vote for him, although I think Melanie would be the most attractive first']
['I do not hate him.']
['Youtube.com also features many of the most funny ads.']
['I enjoy watching my companions attempt to role-play with them.']
['Some of my favorite television series are Monk, the Duke of Hazard, Miami Vice']
['I have a desire to produce videos on Full Metal Alchemist.']
['I would travel to that location and physically assault you at this very moment, however, I am']
['Your mother is so unintelligent that she was hit by a cup and to

In [104]:
print("hello")

hello


In [125]:
import os
import requests

# Define the repository and file names
repo_name = "Pushparaj20/t5-base-finetuned"  # Replace with your actual username and repository name
file_name = "weight.pkl"

# Construct the URL to fetch the file from the repository
file_url = f"https://huggingface.co/{repo_name}/resolve/main/{file_name}"

# Define the directory where you want to save the file locally
output_dir = "logreg"  # Choose a directory name

# Make a GET request to download the file
response = requests.get(file_url)

# Check if the request was successful
if response.status_code == 200:
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    # Write the file content to disk
    with open(os.path.join(output_dir, file_name), "wb") as file:
        file.write(response.content)
    print(f"File '{file_name}' downloaded successfully to '{output_dir}'.")
else:
    print(f"Failed to download '{file_name}' from '{repo_name}'. Status code: {response.status_code}")


File 'weight.pkl' downloaded successfully to 'logreg'.


In [147]:
weights_file


'logreg_weights.pkl'

In [99]:
pipeline

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_df=0.85, max_features=200000,
                                 ngram_range=(1, 3))),
                ('classifier', LogisticRegression(max_iter=5000))])

In [122]:
import joblib

# Load the weights from the file
loaded_weights = joblib.load('logreg_weights.pkl')

# Print the type of loaded weights and the first few entries
print(type(loaded_weights))
print(list(loaded_weights.items())[:10])  # Print the first 10 items


# Check if the structure is similar to the weights dictionary


<class 'dict'>
[('00', -0.31355464842683434), ('00 450', 0.0651834349662771), ('00 450 00', 0.0651834349662771), ('00 92', -0.0008144601026945803), ('00 cad', 0.12968708244671773), ('00 central', -0.0062241793331848745), ('00 central time', -0.0062241793331848745), ('00 eastern', 0.019410039255864337), ('00 eastern time', 0.019410039255864337), ('00 english', -0.02147050676313063)]


In [134]:
import joblib
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import PorterStemmer

# Load the weights from the file
weights_file = 'logreg/weight.pkl'
loaded_weights = joblib.load(weights_file)

# Define a sample sentence
sentence = "s."

# Specify the value of is_formal
is_formal = True  # Example value
stemmer = SnowballStemmer("english")

# Define the function to replace words with POS tags
def replace_with_pos_tags(sentence, weights, is_formal):
    tokens = word_tokenize(sentence)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]  # Stem tokens
    pos_tags = pos_tag(tokens)  # Use original tokens for POS tagging
    replaced_sentence = []

    # Calculate N as described
    N = len(stemmed_tokens) // 4

    # Filter out stop words and stem them
    tokens_without_stopwords = [stemmer.stem(token) for token in tokens if token.lower() not in stop_words]

    # Sort stemmed tokens by weight
    sorted_tokens = sorted(tokens_without_stopwords, key=lambda x: abs(weights.get(x, 0)), reverse=True)

    # Extract N tokens with highest absolute weights
    top_N_tokens = sorted_tokens[:N]

    for token, tag in pos_tags:
        stemmed_token = stemmer.stem(token)
        if stemmed_token in top_N_tokens:
            weight = abs(weights.get(stemmed_token, 0))
            if (is_formal and weight >= 0.01) or (not is_formal and weight >= 0.02):
                replaced_sentence.append(tag)
            else:
                replaced_sentence.append(token)
        else:
            replaced_sentence.append(token)

    return ' '.join(replaced_sentence)

# Call the function with loaded weights, sample sentence, and is_formal value
transformed_sentence = replace_with_pos_tags(sentence, loaded_weights, is_formal)

# Print the transformed sentence
print("Transformed Sentence:", transformed_sentence)


Transformed Sentence: I VBP have a NN that he has a NN on you .


In [128]:
from transformers import AutoModelForSeq2SeqLM

# Load the model from the model hu
model = AutoModelForSeq2SeqLM.from_pretrained("Pushparaj20/t5-base-finetuned")

In [135]:

input_ids = tokenizer(transformed_sentence, return_tensors="pt").input_ids
print(input_ids)

# Generate predictions
output = model.generate(input_ids)
b=output.numpy()
tokenizer.decode(b[0], skip_special_tokens=True)

tensor([[   27,   584, 11165,    43,     3,     9,     3, 17235,    24,     3,
            88,    65,     3,     9,     3, 17235,    30,    25,     3,     5,
             1]])


'I still have a feeling that he has a crush on you.'

In [124]:
weights_file = "weight.pkl"
joblib.dump(weights, weights_file)

['weight.pkl']